# Viewing Data Drift with the Notebook Profile Visualizer

In this notebook, we'll show how you can use whylog's Notebook Profile Visualizer to compare two different sets of the same data. This includes:
- __Data Drift__: Detecting feature drift between two datasets' distributions
- __Data Visualization__: Comparing feature's histograms and bar charts
- __Summary Statistics__: Visualizing Summary Statistics of individual features
- __Data Quality__: Defining feature/dataset constraints and generating reports for Data Quality

# Data Drift on Wine Quality

To demonstrate the Profile Visualizer, let's use [UCI's Wine Quality Dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality), frequently used for learning purposes. Classification is one possible task, where we predict the wine's quality based on its features, like pH, density and percent alcohol content.

In this example, we will split the available dataset in two groups: wines with alcohol content (`alcohol` feature) below and above 11. The first group is considered our baseline (or reference) dataset, while the second will be our target dataset. The goal here is to induce a case of __Sample Selection Bias__, where the training sample is not representative of the population.

The example used here was inspired by the article [A Primer on Data Drift](https://medium.com/data-from-the-trenches/a-primer-on-data-drift-18789ef252a6). If you're interested in more information on this use case, or the theory behind Data Drift, it's a great read!

## Installing Dependencies

In [1]:
# Install these if you don't have them already
# %%sh
# pip install --upgrade pip -q
# pip install whylogs -U -q
# pip install pybars3 -U -q

## Loading the data

In [5]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Disabling false positive warning

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine = pd.read_csv(url,sep=";")
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


We'll split the wines in two groups. The ones with `alcohol` below 11 will form our reference sample, and the ones above 11 will form our target dataset.

In [7]:
cond_reference = (wine['alcohol']<=11)
wine_reference = wine.loc[cond_reference]

cond_target = (wine['alcohol']>11)
wine_target = wine.loc[cond_target]

Let's also add some missing values to `citric acid`, to see how this is reflected in the Profile Visualizer later on.

In [8]:
ixs = wine_target.iloc[100:110].index
wine_target.loc[ixs,'citric acid'] = None

The `quality` feature is a numerical one, representing the wine's quality. Let's tranform it to a categorical feature, where each wine is classified as Good or Bad. Anything above 6.5 is a good a wine. Otherwise, it's bad.

In [9]:
import pandas as pd

bins = (2, 6.5, 8)
group_names = ['bad', 'good']

wine_reference['quality'] = pd.cut(wine_reference['quality'], bins = bins, labels = group_names)
wine_target['quality'] = pd.cut(wine_target['quality'], bins = bins, labels = group_names)


Now, we can profile our dataframes with `whylogs`. We will keep these profiles in-memory to use them later as inputs for the `NotebookProfileVisualizer`

In [10]:
import datetime
from whylogs import get_or_create_session


session = get_or_create_session()
now = datetime.datetime.now()

with session.logger("drift-test", dataset_timestamp=now-datetime.timedelta(days=1)) as logger:
    logger.log_dataframe(wine_reference)
    reference_profile = logger.profile


WARN: Missing config


In [11]:
session = get_or_create_session()
now = datetime.datetime.now()

with session.logger("drift-test", dataset_timestamp=now) as logger:
    logger.log_dataframe(wine_target)
    target_profile = logger.profile

Let's instantiate `NotebookProfileVisualizer` and set the reference and target profiles:

In [12]:
from whylogs.viz import NotebookProfileVisualizer

visualization = NotebookProfileVisualizer()
visualization.set_profiles(target_profile=target_profile, reference_profile=reference_profile)

Now, we're able to generate all sorts of plots and reports.
Let's take a look at some of them.

# Drift Summary

With `summary_drift_report`, we have overview statistics, such as number of observations and missing cells, as well as a comparison between both profile's features, with regards to each feature's distribution, and drift calculations for numerical or categorical features.

The report also displays alerts related to each of the feature's drift severity.

You can also search for a specific feature, or filter by inferred type.

In [13]:
visualization.summary_drift_report()


The drift values are calculated in different ways, depending on the inferred type. For fractional features, the drift detection uses the quantile values and the corresponding CDFs to calculate the approximate KS statistic.For categorical features, the drift detections uses the top frequent items summary, unique count estimate and total count estimate for each feature to calculate the estimated Chi-Squared statistic.

For `alcohol`, there's an alert of severe drift, with calculated p-value of 0.00. That makes sense, since both distributions are mutually exclusive.

We can also conclude some thing just by visually inspecting the distributions. We can clearly see that the "good-to-bad" ratio changes significantly between both profiles. That in itself is a good indicator that the alcohol content might be correlated to the wine's quality

The drift value is also relevant for a number of other features. For example, the `density` also is flagged with significant drift. Let's look at this feature in particular.

# Histograms and Bar charts

Now that we have a general idea of both profiles, let's take a look at some of the individual features.

First, let's use the `double_histogram` to check on the `density` feature.

In [14]:
visualization.double_histogram(feature_names="density")

We can visually assess that there seems to be a drift between both distributions indeed. Maybe the alcohol content plays a significant role on the wine's density.

As is the case with the alcohol content, our potential model would see density values in production never before seen in the training test. We can certainly expect performance degradation during production.

Let's check the `alcohol` feature. Obviously there's a clear separation between the distributions at the value of `11`.

In [15]:
visualization.double_histogram(feature_names="alcohol")

In addition to the histograms, we can also plot distribution charts for categorical variables, like the `quality` feature.

In [16]:
visualization.distribution_chart(feature_names="quality")


We can also look at the difference between distributions:

In [17]:
visualization.difference_distribution_chart(feature_names="quality")


We can see that there is 800 or so more "bads" in the Reference profile, and 50 or so more "goods" on the target profile.

# Feature Statistics

With `feature_statistics`, we have access to very useful statistics by passing the feature and profile name:

In [18]:
visualization.feature_statistics(feature_name="citric acid", profile="target")


Looks like we have 72 distinct values for `citric acid`, ranging from 0 to 0.79. We can also see the 10 missing values injected earlier.

# Constraints

whylogs also enables data quality checks, called `constraints`. This is a topic on its own and will be explored in details in a separate example.

There are a lot of different types of constraints, but for now we'll use a small number of them, as the goal is to show how you can generate a report of the applied constraints through the Profile Visualizer.

Let's set some constraints. For example, we could expect:

1. `quality` should have only `bad` or `good` categories
2. `pH` should never be above 4.5
3.  `volatile acidity` should never be below 0.14
4. The set of features of the profile should match the set of feature of Reference profile

We have different types of constraints. Items #1,#2 and #3 are Summary Constraints, whereas #4 is a Table Shape Constraint. Let's define the specific constraints and assemble them into a DatasetConstraints object. This is a kind of contract that we expect our data to conform.

In [19]:
from whylogs.core.statistics.constraints import (distinctValuesEqualSetConstraint,columnsMatchSetConstraint)
from whylogs.core.statistics.constraints import (SummaryConstraints,DatasetConstraints,maxLessThanEqualConstraint,minGreaterThanEqualConstraint)
column_set = set(wine_reference.columns)
def get_sample_dataset_constraints():
    eq_set = distinctValuesEqualSetConstraint(reference_set={'bad', 'good'})
    max_less_than_equal_value = maxLessThanEqualConstraint(value=4.5)
    min_greater_than_equal_value = minGreaterThanEqualConstraint(value=0.14)
    columns_match = columnsMatchSetConstraint(column_set)
    return DatasetConstraints(
        None,
        summary_constraints={"quality": [eq_set],
                            "pH": [max_less_than_equal_value],
                            "volatile acidity": [min_greater_than_equal_value]},
        table_shape_constraints=[columns_match]
        
    )
dc = get_sample_dataset_constraints()

We pass these constraints as an argument to the logger. Since we didn't do this before, let's log another profile. The difference is that we will now pass the `constraints` argument to the logger object, with the Dataset Constraint.

In [20]:
import datetime
from whylogs import get_or_create_session

session = get_or_create_session()
now = datetime.datetime.now()

with session.logger("constraint-test", dataset_timestamp=now,constraints=dc) as logger:
    logger.log_dataframe(wine_target)
    logger.log({"quality": None})
    # logger.log_dataframe(metrics_target)
    constraints_profile = logger.profile

Constraint of the Summary type need to be applied afterwards:

In [21]:
constraints_profile.apply_summary_constraints()

[('quality', [("distinct values are equal to the set {'bad', 'good'}", 1, 0)]),
 ('pH', [('maximum is less than or equal to 4.5', 1, 0)]),
 ('volatile acidity', [('minimum is greater than or equal to 0.14', 1, 1)])]

Let's set up the NotebookProfileVisualizer once again:

In [22]:
from whylogs.viz import NotebookProfileVisualizer

visualization = NotebookProfileVisualizer()
visualization.set_profiles(target_profile=target_profile)

The report below gives us a summary of the applied constraint, both passing and failing ones.

We can also filter to show only the ones that failes, or only the ones that passed:

In [23]:
visualization.constraints_report(dc)

# Downloading the Visualization Output

All of the previous visualizations can be downloaded in `HTML` format for further inspection. Just run:

In [24]:
import os
os.getcwd()
visualization.download(html=visualization.constraints_report(dc), html_file_name=os.getcwd()+"/example")

We're downloading the constraints report here, but you can simply replace it for your preferred visualization. 

# Reference

- https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

- https://www.kaggle.com/vishalyo990/prediction-of-quality-of-wine

- https://medium.com/data-from-the-trenches/a-primer-on-data-drift-18789ef252a6